# Data Loading and Preprocessing

In [40]:
import pandas as pd
import numpy as np

In [41]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [42]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [43]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [44]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [45]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [46]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [47]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [48]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [49]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# Model 1.

<p>1.Here I am using 40 Lstm cells instead of 32 Lstm cells to see if Performance improves </p>
<p>2. I am keeping Dropout as 0.5 </p>

In [50]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [55]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 40 #tune n_hidden

In [52]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [53]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

E:\anaconda\envs\tf18\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [54]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [56]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 40)                8000      
_________________________________________________________________
dropout_5 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 246       
Total params: 8,246
Trainable params: 8,246
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [58]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 378s 51ms/step - loss: 1.3058 - acc: 0.4382 - val_loss: 1.1812 - val_acc: 0.4941
Epoch 2/30
7352/7352 [==============================] - 334s 45ms/step - loss: 1.0862 - acc: 0.5246 - val_loss: 1.1028 - val_acc: 0.5219
Epoch 3/30
7352/7352 [==============================] - 291s 40ms/step - loss: 1.0036 - acc: 0.5536 - val_loss: 1.0120 - val_acc: 0.5779
Epoch 4/30
7352/7352 [==============================] - 293s 40ms/step - loss: 0.9305 - acc: 0.5774 - val_loss: 0.9723 - val_acc: 0.5568
Epoch 5/30
7352/7352 [==============================] - 285s 39ms/step - loss: 0.8494 - acc: 0.5977 - val_loss: 0.8653 - val_acc: 0.5881
Epoch 6/30
7352/7352 [==============================] - 303s 41ms/step - loss: 0.7722 - acc: 0.6473 - val_loss: 0.7642 - val_acc: 0.6125
Epoch 7/30
7352/7352 [==============================] - 346s 47ms/step - loss: 0.7612 - acc: 0.6772 - val_loss: 0.7530 - val_acc: 

In [59]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      425        58        8                   0   
STANDING                 0      129       399        4                   0   
WALKING                  0        0         0      480                   6   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        0         0       12                   4   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           10  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 455  


In [60]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 17s 6ms/step


In [61]:
score

[0.20021087040396643, 0.9205972175093315]

<p> 1. By increasing Number of Lstm units we have improved , earlier by using 32 Lstm units we got loss of 0.30 and accuray of 90% but now we got loss as 0.20 and accuracy as 92%</p>

# Model 2.

Let us use same 40 Lstm units architecture but now let us regularize our model more , that is instead of keepin dropout rate as 0.6 lets keep dropout rate as 0.8

In [62]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.8))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 40)                8000      
_________________________________________________________________
dropout_6 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 246       
Total params: 8,246
Trainable params: 8,246
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [64]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 296s 40ms/step - loss: 1.4205 - acc: 0.3940 - val_loss: 1.2551 - val_acc: 0.4652
Epoch 2/30
7352/7352 [==============================] - 282s 38ms/step - loss: 1.1377 - acc: 0.5239 - val_loss: 0.9639 - val_acc: 0.6125
Epoch 3/30
7352/7352 [==============================] - 281s 38ms/step - loss: 0.9618 - acc: 0.6007 - val_loss: 0.8383 - val_acc: 0.6549
Epoch 4/30
7352/7352 [==============================] - 277s 38ms/step - loss: 0.8524 - acc: 0.6417 - val_loss: 0.8336 - val_acc: 0.6583
Epoch 5/30
7352/7352 [==============================] - 283s 38ms/step - loss: 0.8055 - acc: 0.6536 - val_loss: 0.7626 - val_acc: 0.6736
Epoch 6/30
7352/7352 [==============================] - 279s 38ms/step - loss: 0.7644 - acc: 0.6714 - val_loss: 0.7726 - val_acc: 0.6512
Epoch 7/30
7352/7352 [==============================] - 281s 38ms/step - loss: 0.7439 - acc: 0.6859 - val_loss: 0.7534 - val_acc: 

In [65]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 517        0        20        0                   0   
SITTING                  0      420        71        0                   0   
STANDING                 0      125       406        1                   0   
WALKING                  0        0         0      443                  35   
WALKING_DOWNSTAIRS       0        0         0        0                 414   
WALKING_UPSTAIRS         0        0         0        4                  25   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           18  
WALKING_DOWNSTAIRS                 6  
WALKING_UPSTAIRS                 442  


In [66]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 19s 7ms/step


In [67]:
score

[0.32103655033073797, 0.8965049202578894]

Here we got 90% accuracy and loss as 0.32 percent this could be because we have regulrized our model a lot . Most probably this is case of underfitting as our dropout rate is too high .

# 3 . Model

<p> 1. Let us try 2 layers Lstm model </p>
<p> 2. To avoid overfitting we should regularized our model to good number by regulating dropout rates but adding too large dropouts also creates bad generalization model typically underfit model</p>

In [73]:
# Initiliazing the sequential model
from keras.layers import Flatten
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer

model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128, 40)           8000      
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 40)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 30726     
Total params: 38,726
Trainable params: 38,726
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [75]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 271s 37ms/step - loss: 0.8029 - acc: 0.5252 - val_loss: 0.7724 - val_acc: 0.6196
Epoch 2/30
7352/7352 [==============================] - 281s 38ms/step - loss: 0.3059 - acc: 0.8742 - val_loss: 0.5498 - val_acc: 0.8409
Epoch 3/30
7352/7352 [==============================] - 287s 39ms/step - loss: 0.1848 - acc: 0.9257 - val_loss: 0.3685 - val_acc: 0.8914
Epoch 4/30
7352/7352 [==============================] - 271s 37ms/step - loss: 0.1543 - acc: 0.9351 - val_loss: 0.3511 - val_acc: 0.8904
Epoch 5/30
7352/7352 [==============================] - 274s 37ms/step - loss: 0.1423 - acc: 0.9422 - val_loss: 0.2856 - val_acc: 0.9046
Epoch 6/30
7352/7352 [==============================] - 272s 37ms/step - loss: 0.1362 - acc: 0.9442 - val_loss: 0.2575 - val_acc: 0.9097
Epoch 7/30
7352/7352 [==============================] - 270s 37ms/step - loss: 0.1324 - acc: 0.9425 - val_loss: 0.3315 - val_acc: 

In [76]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      359       126        0                   0   
STANDING                 0       61       470        1                   0   
WALKING                  0        0         0      462                  30   
WALKING_DOWNSTAIRS       0        0         0        0                 373   
WALKING_UPSTAIRS         0        7         2        8                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                47  
WALKING_UPSTAIRS                 449  


In [77]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 19s 7ms/step


In [78]:
score

[0.49729748394005446, 0.8992195453003053]

<p>Although our model converges faster to best accuracy still we cannot go above 90 % .</p>

# Conclusions

Some of the conclusions are tabulated as given below.

<table style="width:100%">
    <tr>
    <th>Model No</th>
    <th>Number of Lstm Layers</th> 
    <th>Number of Lstm Units</th>
    <th>Dropout Rate</th>
    <th>Loss</th>
    <th>Accuracy</th>
  </tr>
    <tr>
    <th>1</th>
    <th>1</th> 
    <th>32</th>
    <th>0.5</th>
    <th>0.3087</th>
    <th>90%</th>
  </tr>
     <tr>
    <th>2</th>
    <th>1</th> 
    <th>40</th>
    <th>0.5</th>
    <th>0.20</th>
    <th>92%</th>
  </tr>
    <tr>
    <th>3</th>
    <th>1</th> 
    <th>40</th>
    <th>0.8</th>
    <th>0.321</th>
    <th>89.65%</th>
  </tr>
     <tr>
    <th>4</th>
    <th>2</th> 
    <th>40</th>
    <th>0.6</th>
    <th>0.49</th>
    <th>90%</th>
  </tr>
</table>

<p> 1 . Lstms are costly to train. </p>
<p> 2. With Increase in Lstm units the performance of Lstm can improve but it is not guaranteed as we have to keep other things into consideration like dropouts </p>
<p> 3. Dropout rate should be properly regulated otherwise we could easily overfit or underfit</p>
<p> 4. As Number of Layers in Lstm Unit Increases it becomes more expensive to train and hence tuning hyperparameter is not so easy.</p>
<p> 5. With Enough of compute power we can define nice parameter space and Hyper tune our Lstm model </p>